In [ ]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 77 kB 3.4 MB/s 
     |████████████████████████████████| 6.6 MB 64.4 MB/s 
     |████████████████████████████████| 880 kB 7.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=72088d32ec70d24b8b26cbae5816d05693985e626ef3fa2b53fdcf0e20eebb1c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')  
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
from nltk.tag import pos_tag


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import dataset
df=pd.read_csv('gdrive/My Drive/CS230 PROJECT/spotify_songs.csv')

In [ ]:
# Filter songs by english
df_english = df[df['language']=='en']

In [ ]:
# Convert all song lyrics into list of strings
lyrics = df_english.lyrics.to_list()

In [ ]:
# Tokenizer which breaks string into verses based on capital letter
def capitalSplit(lyric):
  return re.findall('[A-Z][^A-Z]*', lyric)
  
def parenComb(verses):
  paren_bool = False
  for idx, x in enumerate(verses):
    if '(' in x:
      if ')' in x:
        continue
      else:
        paren_bool = True
        temp_idx = idx
    if ')' in x and paren_bool == True:
      verses[temp_idx:idx+1] = [''.join(verses[temp_idx:idx+1])]
      paren_bool = False
  return verses

def dealingwithI(verses):
  for idx, x in enumerate(verses):
    if x[0] == 'I':
      if idx > 0 and len(verses[idx - 1].split()) < 7:
        verses[idx-1:idx+1] = [''.join(verses[idx-1:idx+1])]
  return verses

def verseSplit(lyric):
  return dealingwithI(parenComb(capitalSplit(lyric)))

In [ ]:
i = 36

In [ ]:
len(lyrics)

15405

In [ ]:
len(verseSplit(lyrics[i]))

44

In [ ]:
dataset = []
id = 0
for lyric in lyrics: 
  split= verseSplit(lyric)
  # break
  if len(split)>1: 
    for i in range(len(split)-1): 
      curr = {}
      curr["id"] = id
      curr["translation"] = {}
      translation = {}
      translation["en"] = split[i]
      translation["fr"] = split[i+1]
      curr["translation"]= translation
      dataset.append(curr)
      id += 1



In [ ]:
len(dataset)

1064752

In [ ]:
dataset[2]

{'id': 2,
 'translation': {'en': 'And the angels, smiled at me I saw you, in that lonely bench ',
  'fr': 'At half past four, I kissed your soft soft hands and at 6 '}}

In [ ]:
!pip install datasets

     |████████████████████████████████| 342 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 69.4 MB/s 
     |████████████████████████████████| 1.1 MB 25.9 MB/s 
     |████████████████████████████████| 136 kB 54.9 MB/s 
     |████████████████████████████████| 127 kB 56.8 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    # print(model_inputs)
    return model_inputs

In [ ]:
train = dataset[:681442]
test = dataset[681442:851802]
final_test = dataset[851802:]

In [ ]:
len(train)

681442

In [ ]:
len(test)

170360

In [ ]:
import datasets
import pandas as pd
from datasets import Dataset
train_df = pd.DataFrame(train)

test_df = pd.DataFrame(test)

train_dataset = Dataset.from_dict(train_df)
test_dataset = Dataset.from_dict(test_df)
my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

In [ ]:
tokenized_books = my_dataset_dict.map(preprocess_function, batched=True)

  0%|          | 0/682 [00:00<?, ?ba/s]

  0%|          | 0/171 [00:00<?, ?ba/s]

In [ ]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 681442
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 170360
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
len(tokenized_books["train"]) 

681442

In [ ]:
len(tokenized_books["test"])

170360

In [ ]:
shortened_books_train = tokenized_books["train"]

In [ ]:
shortened_books_test = tokenized_books["test"]

In [ ]:
len(shortened_books_train)

170360

In [ ]:
shortened_books_train[0]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'id': 681442,
 'input_ids': [13959, 1566, 12, 2379, 10, 2645, 114, 5234, 1],
 'labels': [2645, 114, 5234, 1],
 'translation': {'en': 'Who like boys ', 'fr': 'Who like boys '}}

In [ ]:
# can take a long time
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # weight_decay=0.01,
    predict_with_generate=True,
    save_total_limit=3,
    num_train_epochs=1,
    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=shortened_books_train,
    eval_dataset=shortened_books_test,
    # train_dataset=tokenized_books["train"],
    # eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 170360
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10648


Epoch,Training Loss,Validation Loss
1,3.466700,3.324517


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=10648, training_loss=3.561760817140139, metrics={'train_runtime': 1202.0095, 'train_samples_per_second': 141.729, 'train_steps_per_second': 8.858, 'total_flos': 1254301292298240.0, 'train_loss': 3.561760817140139, 'epoch': 1.0})

In [ ]:
final_test[0]["translation"]["en"]

"Sky's rainin confetti, singin out the "

In [ ]:
import torch
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model = model.to(device) # the network should be on GPU, next(model.parameters()).is_cuda==False

In [ ]:
# How many friends are you gonna set on fire? How many hearts are gonna end in flames? How many nights are you gonna play vampire Writing a book of crossed out names? I don't want to be alone Thinking about, thinking about the days I don't want to be alone Thinking about the days That you, you were running after me You, you were all the I could see How many lives are you gonna let expire? How many sparks of love have died in vain? How many nights will it take till you grow tired Hunting for the one that got away? I don't want to be alone Thinking about, thinking about the days I don't want to be alone Thinking about the days That you, you were running after me You, you were all the I could see Never never gonna make a home Sitting on a skull mountain throne Limestone, you'll be digging up, digging up the graves Limestone, you'll be digging up the graves That you, you were running after me You, and Louis Michelle Chablis You, you were all that I could see You, were light dancing on the sea

In [ ]:
inputs = tokenizer("How many friends are you gonna set on fire?", return_tensors="pt")
inputs = inputs.to(device)
generation_output = model.generate(**inputs, min_length=100)
print(tokenizer.decode(generation_output[0]))

<pad> How many friends are you gonna set on fire? I'm gonna set on fire


In [ ]:
len(final_test)

212950

In [ ]:
final_test = final_test[:200]

In [ ]:
outputs = []
test_x= []
for i in range(len(final_test)):
  curr = final_test[i]["translation"]["en"]
  test_x.append(curr)
  inputs = tokenizer(curr, return_tensors="pt")
  inputs = inputs.to(device)
  generation_output = model.generate(**inputs, min_length=100)
  y = tokenizer.decode(generation_output[0])
  outputs.append(y)

In [ ]:
df = pd.DataFrame(outputs, columns=["colummn"])
df.to_csv('predicted.csv', index=False)
df = pd.DataFrame(test_x, columns=["colummn"])
df.to_csv('test_x.csv', index=False)

In [ ]:
from nltk import word_tokenize
test_y_true = []
metrics = []
for i in range(len(outputs)):
  y_predicted = outputs[i]
  y_label = final_test[i]["translation"]["fr"]
  test_y_true.append(y_label)
  types1 = set(word_tokenize(y_predicted))
  types2 = set(word_tokenize(y_label))
  lenIntersect = len(set.intersection(types1, types2))
  lenUnion = len(set.union(types1, types2))
  metrics.append(lenIntersect / lenUnion)

In [ ]:
metrics
df = pd.DataFrame(metrics, columns=["colummn"])
df.to_csv('metrics.csv', index=False)
df = pd.DataFrame(test_y_true, columns=["colummn"])
df.to_csv('test_y_true.csv', index=False)

In [ ]:
avg_similarity = np.sum(metrics)/len(metrics)
print(avg_similarity)

0.08013123088152954

In [ ]:
# find the best predictions 
res = 0 
best = metrics[0]
for i in range(len(metrics)): 
  if best < metrics[i]:
    best = metrics[i]
    res = i 
print(best)
print("predicted: "+outputs[res])
print("y_true_label: "+final_test[res]["translation"]["fr"])
print("input: "+final_test[res]["translation"]["en"])

0.5454545454545454
predicted: <pad> I would do it all again, I would do it all again, I would do it all
y_true_label: I would do it all again '
input: And I would do it all again 
